In [1]:
from mobile_base import MobileBaseSDK

In [2]:
mobile_base = MobileBaseSDK('localhost')

In [3]:
mobile_base

<MobileBase host="localhost" battery_level=27.9
        drive mode=CMD_VEL control mode=OPEN_LOOP>

In [3]:
mobile_base.model_version

1.0

In [3]:
mobile_base.control_mode

'OPEN_LOOP'

In [4]:
mobile_base.set_speed(0, 0, -1.5, 0)

In [4]:
mobile_base.emergency_shutdown()

In [3]:
mobile_base.odometry

{'x': -0.007904427126049995,
 'y': -0.004449585918337107,
 'theta': -0.02315443381667137}

In [3]:
mobile_base.go_to(0, 0, 1.4)

In [4]:
mobile_base.reset_odometry()

In [3]:
mobile_base.drive_mode

'CMD_VEL'

In [3]:
mobile_base.control_mode

'OPEN_LOOP'

In [3]:
mobile_base.battery_level

26.7

In [4]:
from mobile_platform_reachy_pb2 import ControlModePossiblities

In [7]:
ControlModePossiblities.keys()

['NONE_CONTROL_MODE', 'OPEN_LOOP', 'PID']

In [3]:
a = 0.0

In [4]:
if not a:
    print('bob')

bob


In [16]:
class Name:
    def __init__(self, name):
        self._name = name
    def yo(self):
        print('Yo')
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, new_name):
        self._name = new_name

In [17]:
bob = Name('bob')

In [9]:
bob.name = 'Robert'

In [10]:
bob.name

'Robert'

In [ ]:
bob.